In [1]:
#def scrapp_page(vetement):
from bs4 import BeautifulSoup
import requests
import urllib.request 
import os
import urllib
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import sqlalchemy
import time

In [2]:
import os
!pip install opencv-python

In [3]:
Id_article = []
clothes_list = list()
mylittledict2  = []
titre_article_list = []
img_source_tag = []
img_source_src = []
couleur_article = []
size_list = []
df = []

In [4]:
words_to_search = ['Hauts & Tee-shirts','Pantalons','Sweats et sweats à capuche','Robes','Manteaux & vestes','Sandales','Chemises','Baskets','Sacs','Bottines','Jupes']

In [5]:
from PIL import Image
import cv2 as cv
import requests
from io import BytesIO
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import pickle

import tensorflow as tf


### DEFINE GRABCUT FUNCTION ###

def cut(img):

    img = cv.resize(img,(224,224))
    
    mask = np.zeros(img.shape[:2],np.uint8)
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    height, width = img.shape[:2]

    rect = (15,10,width-30,height-15)
    cv.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv.GC_INIT_WITH_RECT)
    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    img2 = img*mask2[:,:,np.newaxis]
    img2[mask2 == 0] = (255, 255, 255)
    
    final = np.ones(img.shape,np.uint8)*0 + img2
    
    return mask, final


In [ ]:
from selenium.webdriver.chrome.options import Options
import webbrowser


def title_troubles(inputString):
    inputString = inputString.encode('ascii', 'ignore').decode('ascii').replace(" ", "_").replace("/", "_")
    return inputString
"""
scrapps the current page
:param vetement: string current clothes searched
:return:
"""
words_to_search = ['Chemises','Baskets','Sacs','Bottines','Jupes']
for clothes in words_to_search: 
    try:
        directory = str(clothes)
        parent_dir = "dating/vinted-val"
        path = os.path.join(parent_dir, str(directory))
        os.mkdir(path)
    except OSError:
        print('Error: Creating directory. ' +  directory)
    
    # get soup
    browser = webdriver.Chrome("C:\\Users\\ursu_luw\\Documents\\chromedriver.exe")
    browser.get('https://www.vinted.fr/')
    browser.maximize_window()
    soup = BeautifulSoup(browser.page_source,'html.parser')
    recherche = browser.find_element_by_xpath('//*[@id="search_text"]')
    recherche.send_keys(clothes)
    time.sleep(0.3)
    recherche.send_keys(Keys.ENTER)

    while True:
        soup = BeautifulSoup(browser.page_source,'html.parser')
        image_tags = soup.find_all('img', class_='js-item-thumbnail item-thumbnail lazy-thumbnail __act_as_lazy loaded')
        # print out image urls
        marque_tag = soup.find_all('a', class_='media__image-wrapper')
        size_tag = soup.find_all('a', class_='u-ui-margin-right-small')

        for image_tag in image_tags:
            img_source_tag = image_tag.get('data-item-id')
            clothes_list.append(clothes)
            browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            if str(img_source_tag) in Id_article: 
                continue
            else:
                
                Id_article.append(img_source_tag)
                try :
                    title_article = image_tag.get('alt').strip()
                    title_article = title_troubles(title_article)
                    #title_article = title_article.replace(" ", "_")
                    print(title_article)
                    #Ajouter le titre de l'article
                    titre_article_list.append(title_article)
                except:
                    title_article = None
                try :
                    img_source = image_tag.get('data-src')
                    print(img_source)
                    #Ajouter la source de l'article 
                    img_source_src.append(img_source)

                except:
                    img_source = None
                try:
                    img_source_tag = image_tag.get('data-item-id')
                    print(img_source_tag)
                except:
                    img_source_tag = None
                try:
                    img_color = image_tag.get('style')
                    img_color = img_color.replace("background-color: ", "")
                    print(img_color)

                    #Ajouter la couleur à une liste
                    couleur_article.append(img_color)
                except:
                    couleur_article.append(img_color)
                try: 
                    for marque_article in marque_tag:
                        marque_article = marque_article.get('data-brand')
                        mylittledict = {"marque_article": marque_article}

                        #Ajouter la marque à une liste
                        mylittledict2.append(marque_article)
                        #mycol.insert_one(mylittledict)
                        #Id_article.append(img_source_tag)
                except: 
                    mylittledict2.append(None)
                try: 
                    for size_article in size_tag:
                        size_dict = {"Taille_article": size_article}
                        #Ajouter la marque à une liste
                        size_list.append(size_article)
                        #mycol.insert_one(size_dict)
                        #Id_article.append(img_source_tag)
                except: 
                    size_list.append(None)

                try: 
                    urllib.request.urlretrieve(img_source, str(path+'/'+img_source_tag+'.jpeg'))
                    original = cv.imread(str(path+'/'+img_source_tag+'.jpeg'))
                    original = cv.resize(original,(224,224))
                    mask, final = cut(original)
                    plt.subplot(1,3,3)
                    plt.imshow(cv.cvtColor(final, cv.COLOR_BGRA2RGB))
                    plt.axis('off')
                    # save the image
                    plt.savefig(str(path+'/'+img_source_tag+'.jpeg'), transparent = True, bbox_inches = 'tight', pad_inches = 0)
                except:
                    print(img_source_tag +" n'a pas pu etre imprimer")
                #mydict = { "Id_article": img_source_tag, "titre_article": title_article, "couleur_article": img_color, "source_article": img_source }
                #mycol.insert_one(mydict)


        try:
            #Prendre les informations de la page en cours
            soup = BeautifulSoup(browser.page_source,'html.parser')
            #Extraire l'adresse URL de la page suivante
            next_page = soup.find('a', class_='next_page new-pagination__action')
            new_page = next_page.get('href')
            #Passer à la page suivante.
            browser.get('https://www.vinted.fr'+str(new_page))
            time.sleep(1)
        except AttributeError: #block à exécuter si on se trouve sur la dernière page des résultats
            browser.quit()

            df = pd.DataFrame(
                    {'Id': Id_article,
                     'Titre': titre_article_list,
                     'Source': img_source_src,
                     'Couleur': couleur_article,
                     'Marque': mylittledict2,
                     'Catégorie': clothes_list,
                     'Taille':size_list
                    })
            break

Chemisier_blanc__boutons
https://images.vinted.net/thumbs/310x430/0672d_zHZLx6P6UJbEqJ6F5bjaQjZE.jpeg?1579362584-dc1cbfa1f7c700b9e27a35c847981f23cba6cac6
395348511
#F5F4F3
Chemisier_noir_transparent
https://images.vinted.net/thumbs/310x430/072ab_Bm3MXmY68xmsnL3cRkmFrK88.jpeg?1579698738-bb98d87a46e8fdf55b407f217cc76b8191be0bea
398942035
#B6B6B6


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


Chemisier_femme
https://images.vinted.net/thumbs/310x430/064a0_tcHnaRE1HC3iKXpjhcFU1nvJ.jpeg?1579191527-97964f8f80a6122e6db9d46e8c301c8f9a262c0f
393810623
#C8CACF
_Chemise_en_jean_vintage_Marlboro_Classics_taille_oversize_
https://images.vinted.net/thumbs/310x430/0509e_gMCewXHTzktdb396FZaF3Jsc.jpeg?1580675729-00d147a80c95820444a52a0f2901db13b9a0ca6e
408997169
#CDD5DE
Chemise_en_jean_Jules
https://images.vinted.net/thumbs/310x430/05ba7_q2B59pWnHqu3BisrguwTRjY1.jpeg?1579982311-85b439db49a6c67212f87d804dcacc877f8ab14e
401663397
#CCD0D1
Chemisier_Morgan_Noir
https://images.vinted.net/thumbs/310x430/045a1_BP3QUGht77FYVvNXt5AJ5dA4.jpeg?1571962646-dec2732c238e0f3a7eb24063536590b2ed4aac88
329659870
#C0C0C0
Yves_Saint_Laurent_Shirt
https://images.vinted.net/thumbs/310x430/04fab_Vu3bPv67LNMqpyYszBAXRK4r.jpeg?1580590540-ff858082cda4b46a7948fba0dfabf4fdba451491
407754422
#EAEBEC
Magnifique_chemise_Kenzo
https://images.vinted.net/thumbs/310x430/067be_Zpj8mivGMomdXhqPg787z2Lk.jpeg?1575840193-2bd0b6f

Chemise_XL_vintage
https://images.vinted.net/thumbs/310x430/074dc_P7jmPAjrRHKHL58Uf4JmoBhB.jpeg?1580585369-c3b1c918300776bbc7038ecaa135f5cca47376fa
407697538
#DDC5B5
Chemises_Versace_Classic_V2
https://images.vinted.net/thumbs/310x430/058fa_UNvbWHEk3dkFwDxibxgzKGte.jpeg?1580306739-a8d5d6acfdc69492783c53518cf81fb5ceecbb40
405034806
#D7D8D9
Chemise_Mise_au_Green_3XL
https://images.vinted.net/thumbs/310x430/050f1_SzRMEA5WecoZdPTdf3yncTzN.jpeg?1580462149-6f44383d0d3274e5b6e00584b5f4b9a6ffbeeabd
404551106
#E2D8D7
Chemise_en_jean_G-Star
https://images.vinted.net/thumbs/310x430/05fda_cqXm87LtyVoiiW9EyP2K6M4i.jpeg?1577794920-20be90e1cfb80fa987b3d7b6b358d6ae27b6d43a
380972497
#EDF0F3
Burberry_chemise___Camisa
https://images.vinted.net/thumbs/310x430/0708d_NsscPAyGv2wbmh4sZKpdbJUq.jpeg?1580204118-a274542db3c311f3fef5770b2b986783f83cb556
396376955
#C6D2E6
Chemise_Desigual_blanche_taille_38_bon_tat
https://images.vinted.net/thumbs/310x430/053a8_6SZ4yxc9DhYHvaphS1XxiEwT.jpeg?1580672347-4719c5a379ef

Lot_chemise_taille_M
https://images.vinted.net/thumbs/310x430/06e07_gyba1Z7SffncBFPtk5Htiafr.jpeg?1580722259-08fe70c7ce6997739ddaf072851d5dabc54fd248
409209534
#D7D4D3
chemise
https://images.vinted.net/thumbs/310x430/064fd_S8YU1KXvsj6vgAWUepGVWp5x.jpeg?1580320809-429ced2a039f97323a9ff39e3d710bb8c0bd9532
405305325
#EDE3D8
Chemisier_long_neuf_fuschia,_rose,_caramel_valvetheart_Xl_42_14.
https://images.vinted.net/thumbs/310x430/065ee_RbPprmPNY66rs7WEVNxWQ1k9.jpeg?1580565206-cf9b14c108e21fe8e880c33bd47fdfdc1efd1d66
407304005
#E9DBD8
Sublime_chemise_homme_neuve_taille_M_next_laundered_shirt
https://images.vinted.net/thumbs/310x430/0591b_qLahRg3gpb2uirVC1egvgfhv.jpeg?1579790889-2cdae595e6a857a3c031cf20e07958bdbdf9e14d
399835853
#D4D5D3
Vivian_Westwood_x_LEE_Cowboy_Shirt
https://images.vinted.net/thumbs/310x430/06624_mW1wZgogaY4wtKBaBEEHCLpF.jpeg?1579773315-bfebfad216f15d98327b81dabfabe4285626836c
399581493
#E3D1D4
Chemise_Lacoste
https://images.vinted.net/thumbs/310x430/0663d_RxCV4SJdB6wuC15

Camisa_volante
https://images.vinted.net/thumbs/310x430/05fce_4CkMu51vqi5mhHQir9VymA88.jpeg?1580430820-278e792f3674e26b84e0ebbb73502636d1996a84
406243764
#DEDBD7
Chemise_den_Park
https://images.vinted.net/thumbs/310x430/04ecf_gxW2pERgy1i3AapN3Zo6U5Gg.jpeg?1577696541-965fa328be494a5df8a92145f67abe437e1de4db
380239548
#E4E2DE
Chemise_jean_ikks
https://images.vinted.net/thumbs/310x430/0712b_j3fpS8ki8NMu9wLpgnF6HbJM.jpeg?1580653181-395ae3ed3e42b72b3fa135508186c281c6a35cee
408391590
#C2C3C5
Chemise_Bash_T.2
https://images.vinted.net/thumbs/310x430/02b5e_TifZcNGWacm8B7rohPqGMKHR.jpeg?1580749022-5fb2243c81b5c66030069415aec55b7f2967edd5
206856389
#C9C3BE
Lot_kiabi_neuf
https://images.vinted.net/thumbs/310x430/0713d_4fvHGqgurqJ1JG7vEzu948rC.jpeg?1580748942-eb456d2ea174d5172ecdbe064c6f02a4eb838825
409638780
#DFE2DF
Chemise_Tommy_Hilfiger_taille_S
https://images.vinted.net/thumbs/310x430/052dd_DQCjmQzkDuUaBvuGom6gJzXn.jpeg?1580746461-ddfc9d1de3eae4adb19e3022824b2bab82caf8d8
409598350
#E6E5E5
Cami

Chemisier_vintage
https://images.vinted.net/thumbs/310x430/02c8f_v8cd5F1uy95GpX5bELXDubo7.jpeg?1580657502-f00bdef0d27b63f25544bd0a4ad2fc88bfb83cf3
145193542
#EBE5E2
Dolce_&_Gabbana_blouse_it_mt_42
https://images.vinted.net/thumbs/310x430/06676_LxfZfbY3K2k8uYsN6VrvahHv.jpeg?1578666791-a0f4285bb7a41f35b85342a2e8951acfcc426ebd
388367699
#E2E0E0
Chemise_tunique
https://images.vinted.net/thumbs/310x430/06e5e_tk92dsAsca9HHfYTaZ4boVAx.jpeg?1580726581-cc5340c57e27bb91d3540fd28e3e5b96c1ccd5f3
409275911
#C3C2CC
Camisa_de_Stradivarius
https://images.vinted.net/thumbs/310x430/06cee_9nCWKv3gqkn8D5aRxBEisF4M.jpeg?1580743710-3cb38ea2284a9ec4a810aaccc6cf392d16c335ae
409556102
#E1D7CF
Chemise_double
https://images.vinted.net/thumbs/310x430/05c42_1um6tEhNs9wq6qTkCJZnVETr.jpeg?1580133003-2943681dd765ac4994a142e7114961439d4bf504
403426876
#D5E1DF
Chemise_femme_vintage_bleu_ciel_Giulia_Ferrari
https://images.vinted.net/thumbs/310x430/05903_c71WWsAZUxdbU49TvX2fG2v3.jpeg?1580656733-6038289332afaf4d328f73c062

Chemisette__vichy_Ralph_Lauren
https://images.vinted.net/thumbs/310x430/058df_HrTW14JRDmrV8HLpeCsapyD2.jpeg?1579609206-2db733085418252eec10a2a08ac34c8e0e69d56e
398122150
#CED0D3
Chemisier_tara_jarmon
https://images.vinted.net/thumbs/310x430/05cb8_1XB7aRCcU5afsNrs8W2VwfZc.jpeg?1575957469-b9cedccd100cbbfa6cc13229c2c1b2e68e02b661
369021492
#D0CCCC
Chemise_desigual
https://images.vinted.net/thumbs/310x430/05a91_syAMysBP58ZsLvzj8bBTQWRm.jpeg?1577978646-f5c211955eb5dba81c58dff34a43875d8f630be4
382030736
#DBDDE2
Chemise_en_jean_Abercrombie
https://images.vinted.net/thumbs/310x430/02665_LMLXWJpLr9MPWJ9E9CNFftP4.jpeg?1570382358-d3f7463b5b008bb294e797a96da98849506d0083
311789374
#C5CDD7
Chemise_grise_cargo_t_45_46_rarement_port
https://images.vinted.net/thumbs/310x430/05bdc_PwBB7VzYMd3ELGDNrUYiwrXf.jpeg?1580651346-ef5bcf930445e747f08ec0ece3cec57cb5da6436
408329659
#E1D7D4
Chemisier_pois_kaki_Zara
https://images.vinted.net/thumbs/310x430/05139_UKoBJgfHnC4Z7tvEsQfkJADd.jpeg?1580650887-5f5272482a17

Chemisier
https://images.vinted.net/thumbs/310x430/060b1_E8k9zSBcx4t2XXPUoyBzQruW.jpeg?1580732777-dd11b2e05c588cb086f1eaa0a225e30b9d98b74c
409360277
#D3CFD0
Sur_chemise_motif_pied_de_poule
https://images.vinted.net/thumbs/310x430/052da_THSFkzoYf8wptZJ6fDSLLXdB.jpeg?1580732368-41f3a3dd0086246c16bd655e345d3898e06bbd0e
409354250
#CDB8BB
Sobrecamisa_cuadros_Imperial
https://images.vinted.net/thumbs/310x430/04a2f_YW5vaGwmsDGD592t55DuiGdQ.jpeg?1572180174-1489e5ce501f57014e08eb91d33049eef7730bef
331068792
#D2D1D1
Chemisier__longues_manches
https://images.vinted.net/thumbs/310x430/06f7d_CqbPneMb9BnHLzCiqqgn3x7e.jpeg?1580300605-5ec663137e77bea2df5e896c8f0b042cf996b67b
392647691
#C9C7C7
Chemise__carreaux_rose
https://images.vinted.net/thumbs/310x430/06357_41DkCiU9VHoEjSQq2GAXn9Qy.jpeg?1580149043-588930cc3746babf3170a6e51933ab62702d9fda
403712628
#D4BAC0
Chemise__carreaux_noir_et_blanc
https://images.vinted.net/thumbs/310x430/05264_QdoUc13ZnncDmSXNqxcxuKmv.jpeg?1580300700-c4484a191fbffbb33a04358f